In [1]:
import pandas as pd
df=pd.read_csv('news.csv')
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [2]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [3]:
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
print("Shape of dataset:",df.shape)
print("\nAny null values present:\n",df.isnull().sum())
print("\n Categorical Balance:\n",df['label'].value_counts())

Shape of dataset: (6335, 3)

Any null values present:
 title    0
text     0
label    0
dtype: int64

 Categorical Balance:
 REAL    3171
FAKE    3164
Name: label, dtype: int64


In [5]:
df.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [6]:
df['label'].value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [7]:
df.describe()

,title,text,label
count,6335,6335,6335
unique,6256,6060,2
top,OnPolitics | 's politics blog,"Killing Obama administration rules, dismantlin...",REAL
freq,5,58,3171


Really balanced dataset

Label encoding the last column of dataset

In [8]:
X=df.iloc[:,1]
Y=df.iloc[:,-1]

In [9]:
X

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6335, dtype: object

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test , Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [11]:
Y_train

2402    REAL
1922    REAL
3475    FAKE
6197    REAL
4748    FAKE
        ... 
4931    REAL
3264    REAL
1653    FAKE
2607    FAKE
2732    REAL
Name: label, Length: 5068, dtype: object

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid= TfidfVectorizer(stop_words='english')

train_tfid = tfid.fit_transform(X_train)
test_tfid = tfid.transform(X_test)


In [13]:
from sklearn.linear_model import PassiveAggressiveClassifier
ps_model = PassiveAggressiveClassifier(max_iter=50)
ps_model.fit(train_tfid,Y_train)

PassiveAggressiveClassifier(max_iter=50)

In [14]:
ps_predictions = ps_model.predict(test_tfid)

In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
accuracy_score(Y_test,ps_predictions)

0.9337016574585635

In [16]:
print(confusion_matrix(Y_test,ps_predictions))

[[570  45]
 [ 39 613]]


In [17]:
my_news=['This is a really important question, Lambert says. “I don’t want to be passed along to two or three people,” she says. “I want one person to contact.” There may be specific contact points for different areas, she adds, such as the director of nursing for related questions. However, “I want to know that I can pop into the executive director’s office anytime, ask any question and make any kind of complaint,” she emphasizes. “I want to know that person is available. Because sometimes, you have to go up to that level.""']
test2_tfid = tfid.transform(my_news)

In [18]:
test2_tfid

<1x61713 sparse matrix of type '<class 'numpy.float64'>'
	with 32 stored elements in Compressed Sparse Row format>

In [19]:
ps_model.predict(test2_tfid)

array(['REAL'], dtype='<U4')

In [20]:
again_new_news= ['President obama is not performing good, he is a terrorist']

test3= tfid.transform(again_new_news)

In [21]:
ps_model.predict(test3)

array(['FAKE'], dtype='<U4')

# Pickling the Model File for deployment

In [22]:
import pickle

In [24]:
pickle.dump(ps_model, open('classifier.pkl', 'wb'))

# Let's try to load the pickled model and use it for prediction

In [26]:
pickled_model = pickle.load(open('classifier.pkl', 'rb'))

In [28]:
pickled_model.predict(test3)

array(['FAKE'], dtype='<U4')